# Heartbeat - RNN Sequence Classification

This data set is made available by MIT. It contains data about 9,026 heartbeat measurements. Each row represents a single measurement (captured on a timeline). There are a total of 187 data points (columns) for each measurement. This is a multiclass classification task: predict whether the measurement represents a normal heartbeat or other anomalies. 

## Description of Variables

We will use the **hearbeat.csv** data set for this assignment. Each row represents a single measurement. Columns labeled as T1 from T187 are the values of a measurement on the timeline (there are 187 data points, or columns, in a single measurement). 

The last column is the target variable. It shows the label (category) of the measurement as follows:<br>
0 = Normal<br>
1 = Supraventricular premature beat<br>
2 = Premature ventricular contraction<br>
3 = Fusion of ventricular and normal beat<br>
4 = Unclassifiable beat

## Goal

Use the data set **hearbeat.csv** to predict the column called **Target**. The input variables are columns labeled as **T1 to T187**. 

# Important note:

Looks like the original length of the sequences were not the same in the data set. For example, one sequence (i.e., row) had 187 columns, while another one had 150 only. Therefore, the creators of the data set did a "zero padding". This means that they added zeros at end of each sequence to make them the same length. This way, all sequences (i.e., rows) were made to have 187 columns.

If we don't account for the zero-padding, we will create biased models. To account for this, we can use the following as your first layer of your models:<br>
`tf.keras.layers.Masking(mask_value=0, input_shape=[n_steps, n_inputs])`<br>
(Of course, you need to enter your own values for n_steps and n_inputs). After we add this layer, we continue with other layers as usual.

# Read and Prepare the Data

In [1]:
# Insert as many cells as you need for data prep
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [2]:
data = pd.read_csv("heartbeat.csv")

In [3]:
data.shape

(9026, 188)

In [4]:
data.head()

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T179,T180,T181,T182,T183,T184,T185,T186,T187,Target
0,0.987,0.892,0.461,0.1130,0.1490,0.1900,0.1650,0.1620,0.1470,0.1380,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000,0.918,0.621,0.1330,0.1050,0.1250,0.1170,0.0898,0.0703,0.0781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000,0.751,0.143,0.1040,0.0961,0.0519,0.0442,0.0416,0.0364,0.0857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.000,0.740,0.235,0.0464,0.0722,0.0567,0.0103,0.0155,0.0284,0.0155,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.000,0.626,0.276,0.3250,0.4310,0.3900,0.3940,0.3580,0.3740,0.3620,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
y = data['Target']
x = data.drop('Target', axis=1)

In [6]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

In [7]:
#Target variables need to be an array with integer type
train_y = np.array(train_y)
test_y = np.array(test_y)

train_y = train_y.astype(np.int32)
test_y = test_y.astype(np.int32)


In [8]:
#Check the first 10 values of the train_y data set
train_y[0:10]

array([0, 2, 0, 3, 0, 0, 2, 2, 2, 2])

In [9]:
#Convert input variables to a 2-D array with float data type
train_x= np.array(train_x)
test_x= np.array(test_x)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [10]:
train_x

array([[0.973 , 0.927 , 0.367 , ..., 0.    , 0.    , 0.    ],
       [0.988 , 1.    , 0.954 , ..., 0.    , 0.    , 0.    ],
       [0.945 , 0.595 , 0.0199, ..., 0.    , 0.    , 0.    ],
       ...,
       [0.88  , 0.967 , 0.908 , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.0207, 0.022 , ..., 0.    , 0.    , 0.    ],
       [0.993 , 1.    , 0.778 , ..., 0.    , 0.    , 0.    ]],
      dtype=float32)

In [11]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))


In [12]:
train_x.shape, train_y.shape

((6318, 187, 1), (6318,))

In [13]:
train_x

array([[[0.973 ],
        [0.927 ],
        [0.367 ],
        ...,
        [0.    ],
        [0.    ],
        [0.    ]],

       [[0.988 ],
        [1.    ],
        [0.954 ],
        ...,
        [0.    ],
        [0.    ],
        [0.    ]],

       [[0.945 ],
        [0.595 ],
        [0.0199],
        ...,
        [0.    ],
        [0.    ],
        [0.    ]],

       ...,

       [[0.88  ],
        [0.967 ],
        [0.908 ],
        ...,
        [0.    ],
        [0.    ],
        [0.    ]],

       [[0.    ],
        [0.0207],
        [0.022 ],
        ...,
        [0.    ],
        [0.    ],
        [0.    ]],

       [[0.993 ],
        [1.    ],
        [0.778 ],
        ...,
        [0.    ],
        [0.    ],
        [0.    ]]], dtype=float32)

In [14]:
data['Target'].value_counts()/len(data)

0.0    0.581875
4.0    0.178152
2.0    0.160425
1.0    0.061600
3.0    0.017948
Name: Target, dtype: float64

# LSTM Model:Model 1

### Make sure to add the masking layer to the model!

In [41]:
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [42]:
n_steps = 36
n_inputs = 1

model = tf.keras.models.Sequential([
    tf.keras.layers.Masking(mask_value=0, input_shape=[n_steps, n_inputs]),
    tf.keras.layers.LSTM(10, input_shape=[n_steps, n_inputs]),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [43]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(lr=0.001)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=50,
                   validation_data = (test_x, test_y)
                    ,callbacks=callback)

Epoch 1/50
198/198 [==============================] - 23s 88ms/step - loss: 1.2795 - accuracy: 0.5546 - val_loss: 1.1406 - val_accuracy: 0.5794
Epoch 2/50
198/198 [==============================] - 18s 90ms/step - loss: 1.1093 - accuracy: 0.5829 - val_loss: 1.1003 - val_accuracy: 0.5794
Epoch 3/50
198/198 [==============================] - 22s 109ms/step - loss: 1.0902 - accuracy: 0.5829 - val_loss: 1.0981 - val_accuracy: 0.5794
Epoch 4/50
198/198 [==============================] - 22s 109ms/step - loss: 1.1260 - accuracy: 0.5828 - val_loss: 1.1533 - val_accuracy: 0.5794
Epoch 5/50
198/198 [==============================] - 24s 121ms/step - loss: 1.1034 - accuracy: 0.5981 - val_loss: 1.0963 - val_accuracy: 0.6067
Epoch 6/50
198/198 [==============================] - 27s 134ms/step - loss: 1.1161 - accuracy: 0.5801 - val_loss: 1.1115 - val_accuracy: 0.5816
Epoch 7/50
198/198 [==============================] - 24s 120ms/step - loss: 1.0914 - accuracy: 0.5776 - val_loss: 1.0724 - val_accu

In [44]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[1.478166937828064, 0.496676504611969]

In [45]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 1.48
accuracy: 49.67%


## LSTM One Layer: Model 2

In [31]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.Masking(mask_value=0, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(5, activation='softmax' , input_shape=[n_steps, n_inputs])
])

In [32]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(lr=0.001)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=50,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/50
198/198 [==============================] - 18s 79ms/step - loss: 1.4535 - accuracy: 0.5825 - val_loss: 1.3341 - val_accuracy: 0.5794
Epoch 2/50
198/198 [==============================] - 18s 91ms/step - loss: 1.2725 - accuracy: 0.5829 - val_loss: 1.2366 - val_accuracy: 0.5794
Epoch 3/50
198/198 [==============================] - 21s 104ms/step - loss: 1.2128 - accuracy: 0.5829 - val_loss: 1.1966 - val_accuracy: 0.5794
Epoch 4/50
198/198 [==============================] - 20s 102ms/step - loss: 1.1782 - accuracy: 0.5829 - val_loss: 1.1670 - val_accuracy: 0.5794
Epoch 5/50
198/198 [==============================] - 21s 107ms/step - loss: 1.1517 - accuracy: 0.5828 - val_loss: 1.1426 - val_accuracy: 0.5772
Epoch 6/50
198/198 [==============================] - 22s 109ms/step - loss: 1.1307 - accuracy: 0.5744 - val_loss: 1.1227 - val_accuracy: 0.5783
Epoch 7/50
198/198 [==============================] - 21s 104ms/step - loss: 1.1155 - accuracy: 0.5788 - val_loss: 1.1106 - val_accu

In [33]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[1.024549126625061, 0.6111521124839783]

In [34]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 1.02
accuracy: 61.12%


In [35]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 1.02
accuracy: 61.12%


# GRU Model

### Make sure to add the masking layer to this model too!

In [46]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.Masking(mask_value=0, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(20, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(5, activation='softmax')
])

In [47]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(lr=0.001)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=50,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/50
198/198 [==============================] - 96s 411ms/step - loss: 1.1783 - accuracy: 0.5703 - val_loss: 1.0789 - val_accuracy: 0.5569
Epoch 2/50
198/198 [==============================] - 95s 479ms/step - loss: 1.0200 - accuracy: 0.6102 - val_loss: 0.9775 - val_accuracy: 0.6403
Epoch 3/50
198/198 [==============================] - 87s 441ms/step - loss: 0.8359 - accuracy: 0.7229 - val_loss: 0.7664 - val_accuracy: 0.7397
Epoch 4/50
198/198 [==============================] - 91s 460ms/step - loss: 0.6841 - accuracy: 0.7863 - val_loss: 0.6954 - val_accuracy: 0.7862
Epoch 5/50
198/198 [==============================] - 111s 561ms/step - loss: 0.6420 - accuracy: 0.8052 - val_loss: 0.6900 - val_accuracy: 0.7836
Epoch 6/50
198/198 [==============================] - 85s 431ms/step - loss: 0.6094 - accuracy: 0.8151 - val_loss: 0.6492 - val_accuracy: 0.8058
Epoch 7/50
198/198 [==============================] - 100s 506ms/step - loss: 0.5766 - accuracy: 0.8262 - val_loss: 0.6146 - val_

In [48]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.3604240119457245, 0.8892171382904053]

In [49]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.36
accuracy: 88.92%


### Baseline Accuracy 

In [40]:
data['Target'].value_counts()/len(data)

0.0    0.581875
4.0    0.178152
2.0    0.160425
1.0    0.061600
3.0    0.017948
Name: Target, dtype: float64

# Discussion

Briefly answer the following questions: (2 points) 
1) Which model performs the best (and why)?<br>
2) What is the baseline value?<br>
3) Does the best model perform better than the baseline (and why)?<br>
4) Does the best model exhibit any overfitting; what did you do about it?